In [107]:
from bs4 import BeautifulSoup
import requests
import re
import csv
import string
import urllib
import urllib.request
import shutil
import os.path
from datetime import date

In [108]:
def Genes_Genomes_Genetics(from_year):
    
    # My choice of Jornal for the project
    main_url = 'https://www.g3journal.org'
    
    # The List that will be a list of dictionaries and contain all the extracted data 
    result = []
    
    # Specifying the Header columns that we want in the result set
    fieldnames = ['ID', 'AUTHORS', 'PUBLISH DATE', 'TITLE', 'ABSTRACT', 'FULL PAPER','AFFILIATIONS', 'CORRESPONDENCE AUTHOR', 'KEYWORDS', 'CORRESPONDENCE AUTHOR\'S E-MAIL']
    
    # Since we know the Journal started from 2011, this will be used to create the URL for Journal by year
    start_year = from_year - 2010
    end_year = date.today().year - 2010 + 1

    for year in range(start_year,end_year):
    # To control for how many years we want to extract the articles
    #for year in range(start_year,start_year+2):



        # Creating YEAR Folder and path to store html pages
        my_folder_year = os.path.join('C:\\Users\\neha\\NJIT\\R\\HTML\\', str(year+2010))
        if not os.path.exists(my_folder_year):
            os.makedirs(my_folder_year)


        # All months in the year or can control the months needed
        #for i in range(12,13):        
        for i in range(1,13):
        
            # Creating URL for the Journal by Year and Month
            url = main_url+'/content/' + str(year) + '/' +str(i)
            req=requests.get(url)
            soup = BeautifulSoup(req.text, "lxml")
            
            # Getting all Article Links for a Year and Month
            article_links = soup.find_all('div',{'class':'highwire-cite highwire-cite-highwire-article highwire-citation-genetics-list-complete clearfix'})


            # Looping through each Article for a month in a year
            for idx,article in enumerate(article_links):
                total_links = len(article_links)

                art_dict = {}

                # To Control if we do not want to get all the links from the Page
                #if idx in range(0,2):
                if idx in range(0,total_links):


                    art_dict['ID']=idx+1

                    # Getting all Authors
                    if article.find_all('span',{'class':'highwire-citation-author'}) is not None:
                        other_authors = article.find_all('span',{'class':'highwire-citation-author'})
                        for auth_id,auth in enumerate(other_authors):
                            art_dict.setdefault('AUTHORS',[]).append(auth.text)


                    # URL Link
                    art_url = article.find('a',{'class':'highwire-cite-linked-title'}).get('href')
                    if re.findall('\Ahttp',art_url)==[]:
                        art_url=main_url+art_url


                    # Creating Month Folder and path to store html pages
                    my_folder_month = os.path.join(my_folder_year, str(i))
                    if not os.path.exists(my_folder_month):
                        os.makedirs(my_folder_month)


                    # Actual html page is in f variable below
                    if article.find('span',{'class':'highwire-cite-metadata-doi highwire-cite-metadata'}) is not None:
                        htm = article.find('span',{'class':'highwire-cite-metadata-doi highwire-cite-metadata'}).text
                        head , sep, art_htm = htm.partition('https://')
                        x = re.sub(r'/', '.', art_htm) + ".html"
                        f = urllib.request.urlretrieve(art_url,x)
                        shutil.move('C:\\Users\\neha\\NJIT\\R\\'+x, os.path.join(my_folder_month, x))


                    # TITLE of the Article
                    art_text = article.find('a',{'class':'highwire-cite-linked-title'}).text



                    # Content of Link
                    req_art =  requests.get(art_url)
                    soup_art = BeautifulSoup(req_art.text, "lxml")

                    # Full text
                    if soup_art.find("div",{'class':'article fulltext-view'}) is not None:
                        art_content = soup_art.find("div",{'class':'article fulltext-view'})
                        if art_content.find_all('p') is not None:
                            p = art_content.find_all('p')
                            content = ' '.join(item .text for item in p)
                            art_content = content.encode('utf8','replace')


                    # ABSTRACT
                    abs_content = soup_art.find("div",{'class':'section abstract'})
                    art_content_abs = soup_art.find("div",{'class':'article fulltext-view'})
                    abs_content_first = ''
                    abs_content_other = ''
                    if abs_content is not None:
                        q = abs_content.find_all('p')
                        content_first = ' '.join(item .text for item in q)
                        abs_content_first = content_first.encode('utf8','replace')

                        p = art_content_abs.find_all('p',recursive=False)
                        content_other = ' '.join(item .text for item in p)
                        abs_content_other = content_other.encode('utf8','replace')



                    # KEYWORDS
                    if soup_art.find_all("li",{'class':'kwd'}) is not None:
                        art_keywords = soup_art.find_all("li",{'class':'kwd'})
                        for word in art_keywords:
                            art_dict.setdefault('KEYWORDS',[]).append(word.text)

                    # Going to new url now
                    new_url=art_url+'.article-info'
                    req_info =  requests.get(new_url)
                    soup_info = BeautifulSoup(req_info.text, "lxml")

                    art_info = soup_info.find('div',{'id':'mini-panel-jnl_genetics_art_info'})
                    auth_info = soup_info.find('div',{'id':'content-block-markup'})


                    # Published Date 
                    if art_info.find('li',{'class':'published'}) is not None:
                        art_pub = art_info.find('li',{'class':'published'}).text
                        head , sep, art_pub = art_pub.partition('online ')


                    # EMail
                    if auth_info.find_all('span',{'class':'em-link'}) is not None:
                        auth_emails = auth_info.find_all('span',{'class':'em-link'})
                        for email in auth_emails:
                            email_text = re.sub('{at}','@',email.text)
                            art_dict.setdefault('CORRESPONDENCE AUTHOR\'S E-MAIL',[]).append(email_text)

                    # Corresponding Author
                    if auth_info.find_all('li',{'class':'corresp'}) is not None:
                        auth_corr = auth_info.find_all('li',{'class':'corresp'})
                        for corr in auth_corr:
                            corr_text, sep, tail = corr.text.partition('E-mail')
                            corr_text = corr_text.lstrip('â†µ1234567890†↵*')
                            art_dict.setdefault('CORRESPONDENCE AUTHOR',[]).append(corr_text)


                    # Affiliations
                    if auth_info.find_all('li',{'class':'aff'}) is not None:
                        auth_affilitiations = auth_info.find_all('li',{'class':'aff'})
                        for affi in auth_affilitiations:
                            art_dict.setdefault('AFFILIATIONS',[]).append(affi.text.lstrip('â†µ1234567890†↵*'))

                    #wrap extracted data in dictionary 
                    art_dict['PUBLISH DATE'] = art_pub
                    art_dict['TITLE'] = art_text
                    art_dict['ABSTRACT'] = abs_content_first+abs_content_other
                    art_dict['FULL PAPER'] = art_content
                    
                    
                    result.append(art_dict)
    
    # Writing all the information into a csv file with all the headers defined right at the start
    with open('G3_GENOMES.csv', 'w', encoding='utf8', newline='') as output_file:
        fc = csv.DictWriter(output_file, 
                            fieldnames,

                           )
        fc.writeheader()
        fc.writerows(result)

In [109]:
# Calling the Function by passing in Year starting which the Articles will be extracted upto current year(2020)
Genes_Genomes_Genetics(2020)